In [2]:
pip install torch

     -------------------------------------- 172.4/172.4 MB 7.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install torchaudio

     ---------------------------------------- 2.1/2.1 MB 2.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [14]:
import torch
import torch.nn as nn
from torchvision.models import resnet18
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import torch.nn.init as init
import numpy as np
from sklearn.metrics import label_ranking_average_precision_score, top_k_accuracy_score

# Paths to the train, test, and validation data folders
train_dir = 'split_data/train'
test_dir = 'split_data/test'
val_dir = 'split_data/val'

# Image dimensions
img_width, img_height = 224, 224

# Batch size
batch_size = 64

# Number of input channels
input_channels = 1

# Data transformations
data_transforms = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.Grayscale(num_output_channels=input_channels),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize to [-1, 1]
])

# Data loaders
train_dataset = torchvision.datasets.ImageFolder(train_dir, transform=data_transforms)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)

test_dataset = torchvision.datasets.ImageFolder(test_dir, transform=data_transforms)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

val_dataset = torchvision.datasets.ImageFolder(val_dir, transform=data_transforms)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

# Model initialization
model = resnet18(num_classes=len(train_dataset.classes))

# Modify the first convolutional layer to accept 1 input channel
model.conv1 = nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Weight initialization
for m in model.modules():
    if isinstance(m, nn.Conv2d):
        init.normal_(m.weight, mean=0, std=0.02)
    elif isinstance(m, nn.BatchNorm2d):
        init.normal_(m.weight, mean=1, std=0.02)
        init.constant_(m.bias, 0)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
device = torch.device("cpu")
model.to(device)

# Training configuration
epochs = 15

# Train the model
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct / total

    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch+1, epochs, epoch_loss, epoch_acc*100))

# Evaluation
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Convert predictions to one-hot encoded format
y_true = torch.tensor(y_true)
y_pred = torch.tensor(y_pred)
y_pred_onehot = torch.zeros(y_pred.size(0), len(test_dataset.classes))
y_pred_onehot.scatter_(1, y_pred.view(-1, 1), 1)

# Calculate accuracy and loss
accuracy = (y_true == y_pred).sum().item() / len(y_true)
loss = criterion(outputs, labels).item()

print('Accuracy: {:.2f}%'.format(accuracy * 100))
print('Loss:', loss)

# Calculate LWLRAP score
lwlrAP = label_ranking_average_precision_score(y_true.cpu().numpy(), y_pred_onehot.numpy())
print('LWLRAP score:', lwlrAP)

# Calculate top-5 categorical accuracy
top5_accuracy = top_k_accuracy_score(y_true.cpu().numpy(), y_pred_onehot.numpy(), k=5)
print('Top-5 categorical accuracy:', top5_accuracy)


Epoch [1/15], Loss: 2.2689, Accuracy: 23.75%
Epoch [2/15], Loss: 1.6934, Accuracy: 47.99%
Epoch [3/15], Loss: 1.2317, Accuracy: 61.54%
Epoch [4/15], Loss: 1.0334, Accuracy: 68.23%
Epoch [5/15], Loss: 0.9259, Accuracy: 71.33%
Epoch [6/15], Loss: 0.8456, Accuracy: 73.27%
Epoch [7/15], Loss: 0.7656, Accuracy: 76.86%
Epoch [8/15], Loss: 0.7102, Accuracy: 78.34%
Epoch [9/15], Loss: 0.6633, Accuracy: 79.08%
Epoch [10/15], Loss: 0.6040, Accuracy: 81.72%
Epoch [11/15], Loss: 0.5686, Accuracy: 82.68%
Epoch [12/15], Loss: 0.5259, Accuracy: 84.01%
Epoch [13/15], Loss: 0.4773, Accuracy: 85.80%
Epoch [14/15], Loss: 0.4402, Accuracy: 86.47%
Epoch [15/15], Loss: 0.4003, Accuracy: 88.20%
Accuracy: 57.51%
Loss: 0.1699233055114746


ValueError: y_true and y_score have different shape

In [16]:
import torch.nn.functional as F

# Convert predictions to one-hot encoded format
y_true = torch.tensor(y_true)
y_pred = torch.tensor(y_pred)
y_pred_onehot = torch.zeros(y_pred.size(0), len(test_dataset.classes))
y_pred_onehot.scatter_(1, y_pred.view(-1, 1), 1)

# Convert y_true to one-hot encoded format and then numpy array
y_true_onehot = F.one_hot(y_true.squeeze(), num_classes=len(test_dataset.classes)).numpy()

# Calculate LWLRAP score
lwlrAP = label_ranking_average_precision_score(y_true_onehot, y_pred_onehot.numpy())
print('LWLRAP score:', lwlrAP)

# Calculate top-5 categorical accuracy
top5_accuracy = top_k_accuracy_score(y_true, y_pred_onehot.numpy(), k=5)
print('Top-5 categorical accuracy:', top5_accuracy)


LWLRAP score: 0.6033994334277607
Top-5 categorical accuracy: 0.7195467422096318


C:\Users\arham\AppData\Local\Temp\ipykernel_24148\1647601717.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred = torch.tensor(y_pred)
